+ 1. Провести анализ выбранного набора данных. Какова проблемная область?
+ 2. Провести анализ содержимого набора данных. Что является объектом/объектами наблюдения? Каковы атрибуты объектов? Есть ли связи между объектами?
+ 3. Определить бизнес-цель/цели. Каков эффект для бизнеса?
+ 4. Определить цели технического проекта для каждой выделенной ранее бизнес-цели. Что поступает на вход, что является целевым признаком?
+ 5. Определить проблемы выбранного набора данных: зашумленность, смещение, актуальность, выбросы, просачивание данных.
+ 6. Привести примеры решения обнаруженных проблем.
+ 7. Оценить качество набора данных: информативность, степень покрытия, соответствие реальным данным, согласованность меток.
+ 8. Устранить проблему пропущенных данных: удаление, подстановка константного значения (0 или подобное), подстановка среднего значения.
+ 9. Выполнить разбиение набора данных на обучающую, контрольную (при необходимости) и тестовую выборки.
+ 10. Оценить сбалансированность выборок. Оценить необходимость использования методов приращения (аугментации) данных.
+ 11. Выполнить приращение данных методами выборки с избытком (oversampling) и выборки с недостатком (undersampling). Должны быть представлены примеры реализации обоих методов.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

df = pd.read_excel("data/India_tourist_places.xlsx")

# df.info()
display(df.Type.value_counts())
df.head()


Type
Temple           57
Beach            25
Fort             21
Lake             15
National Park    14
                 ..
Village           1
Island            1
Township          1
Orchard           1
Gravity Hill      1
Name: count, Length: 75, dtype: int64

,Zone,State,City,Name,Type,EstablishmentYear,TimeNeededToVisitInHrs,GoogleReviewRating,EntranceFeeInINR,AirportWith50kmRadius,WeeklyOff,Significance,DSLRAllowed,NumberOfGoogleReviewInLakhs,BestTimeToVisit,ImageURL
0,Western,Gujarat,Rann of Kutch,Rann Utsav,Cultural,Unknown,3.0,4.9,7500,Yes,NaN,Cultural,Yes,0.10,Evening,https://www.rannutsav.com/blog/wp-content/uplo...
1,Northern,Punjab,Amritsar,Golden Temple (Harmandir Sahib),Religious Site,1604,1.5,4.9,0,Yes,NaN,Spiritual,Yes,1.90,All,https://upload.wikimedia.org/wikipedia/commons...
2,Northern,Ladakh,Leh,Pangong Tso,Lake,Unknown,2.0,4.9,20,Yes,NaN,Nature,Yes,0.15,Morning,https://dynamic-media-cdn.tripadvisor.com/medi...
3,Western,Maharastra,Mumbai,Siddhivinayak Temple,Temple,1881,2.0,4.8,0,Yes,NaN,Religious,No,1.05,All,https://upload.wikimedia.org/wikipedia/commons...
4,Western,Gujarat,Somnath,Somnath Temple,Temple,1951,2.0,4.8,0,No,NaN,Religious,No,0.39,Morning,https://upload.wikimedia.org/wikipedia/commons...


In [ ]:
display(df.isnull().sum())
display()
display(df.isnull().any())
display()

for i in df.columns:
    null_rate = df[i].isnull().sum() / len(df) * 100
    if null_rate > 0:
        display(f"{i} процент пустых значений: %{null_rate:.2f}")

df = df.drop(["WeeklyOff", "EstablishmentYear"], axis=1)
display(df.head())

9. Выполнить разбиение набора данных на обучающую, контрольную (при необходимости) и тестовую выборки.

In [52]:
from src.utils import split_stratified_into_train_val_test


display(df.DSLRAllowed.value_counts())
display()

data = df[["Zone", "State", "City", "Type", "TimeNeededToVisitInHrs", "GoogleReviewRating", "EntranceFeeInINR", "AirportWith50kmRadius", "Significance", "DSLRAllowed", "NumberOfGoogleReviewInLakhs"]].copy()

df_train, df_val, df_test, y_train, y_val, y_test = split_stratified_into_train_val_test(
   data, stratify_colname="DSLRAllowed", frac_train=0.60, frac_val=0.20, frac_test=0.20
)

display("Обучающая выборка: ", df_train.shape)
display(df_train.DSLRAllowed.value_counts())

display("Контрольная выборка: ", df_val.shape)
display(df_val.DSLRAllowed.value_counts())

display("Тестовая выборка: ", df_test.shape)
display(df_test.DSLRAllowed.value_counts())



DSLRAllowed
Yes    244
No      58
Name: count, dtype: int64

'Обучающая выборка: '

(181, 11)

DSLRAllowed
Yes    146
No      35
Name: count, dtype: int64

'Контрольная выборка: '

(60, 11)

DSLRAllowed
Yes    49
No     11
Name: count, dtype: int64

'Тестовая выборка: '

(61, 11)

DSLRAllowed
Yes    49
No     12
Name: count, dtype: int64

In [ ]:
# Приращение данных методами выборки с избытком (oversampling) и выборки с недостатком (undersampling). Должны быть представлены примеры реализации обоих методов.
# Подготовка

X = df[["Zone", "State", "City", "Type", "TimeNeededToVisitInHrs", 
        "GoogleReviewRating", "EntranceFeeInINR", "AirportWith50kmRadius", 
        "Significance", "NumberOfGoogleReviewInLakhs"]].copy()
y = df["DSLRAllowed"]

X_encoded = pd.get_dummies(X)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [74]:
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_encoded, y_encoded)

X_smote_df = pd.DataFrame(X_smote, columns=X_encoded.columns)
y_smote_series = pd.Series(y_smote, name='DSLRAllowed')
y_smote_labels = le.inverse_transform(y_smote_series)

df_smote = pd.concat([X_smote_df, pd.Series(y_smote_labels, name='DSLRAllowed')], axis=1)

display(df_smote.head())
display(df_smote.DSLRAllowed.value_counts())

,TimeNeededToVisitInHrs,GoogleReviewRating,EntranceFeeInINR,NumberOfGoogleReviewInLakhs,Zone_Central,Zone_Eastern,Zone_North Eastern,Zone_Northern,Zone_Southern,Zone_Western,...,Significance_Nature,Significance_Recreational,Significance_Religious,Significance_Scenic,Significance_Scientific,Significance_Shopping,Significance_Spiritual,Significance_Sports,Significance_Wildlife,DSLRAllowed
0,3.0,4.9,7500,0.10,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,Yes
1,1.5,4.9,0,1.90,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,Yes
2,2.0,4.9,20,0.15,False,False,False,True,False,False,...,True,False,False,False,False,False,False,False,False,Yes
3,2.0,4.8,0,1.05,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,No
4,2.0,4.8,0,0.39,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,No


DSLRAllowed
Yes    244
No     244
Name: count, dtype: int64

In [70]:
rus = RandomUnderSampler(random_state=42)
X_under, y_under = rus.fit_resample(X_encoded, y_encoded)

print(pd.Series(y_under).value_counts())

X_under_df = pd.DataFrame(X_under, columns=X_encoded.columns)
y_under_series = pd.Series(y_under, name="DSLRAllowed")
y_under_labels = le.inverse_transform(y_under_series)

df_under = pd.concat([X_under_df, pd.Series(y_under_labels, name='DSLRAllowed')], axis=1)

display(df_under.head())
display(df_under.DSLRAllowed.value_counts())


0    58
1    58
Name: count, dtype: int64


,TimeNeededToVisitInHrs,GoogleReviewRating,EntranceFeeInINR,NumberOfGoogleReviewInLakhs,Zone_Central,Zone_Eastern,Zone_North Eastern,Zone_Northern,Zone_Southern,Zone_Western,...,Significance_Nature,Significance_Recreational,Significance_Religious,Significance_Scenic,Significance_Scientific,Significance_Shopping,Significance_Spiritual,Significance_Sports,Significance_Wildlife,DSLRAllowed
3,2.0,4.8,0.0,1.05,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,No
4,2.0,4.8,0.0,0.39,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,No
7,1.0,4.8,0.0,0.90,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,No
8,1.5,4.8,0.0,1.20,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,No
13,1.0,4.8,0.0,0.29,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,No


DSLRAllowed
No     58
Yes    58
Name: count, dtype: int64